# Ensemble Tweets - v3

## Changes:

# Tried different models for the Level-2 Ensemble model

# Gradient Boosting Classifier is having the highest F1 score

In [1]:
import pandas as pd

In [2]:
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')


In [3]:
x2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_probas.csv') # Train : Set-2 Predict_probas
x3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_probas.csv')# Val : Set-3 Predict_probas
test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_probas.csv')

Bert2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_Bert_proba.csv')
Bert3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_Bert_proba.csv')
Bert_test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_data_Bert_proba.csv')

Roberta2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_Roberta_proba.csv')
Roberta3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_Roberta_proba.csv')
Roberta_test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_data_Roberta_proba.csv')


In [4]:
Bert_test[:2]

,Bert_1,Bert_0
0,0.954075,0.045925
1,0.965994,0.034006


In [5]:
#Adding Bert Probas to x2, x3, test

x2['Bert_1'] = Bert2['Bert_1']
x2['Bert_0'] = Bert2['Bert_0']

x3['Bert_1'] = Bert3['Bert_1']
x3['Bert_0'] = Bert3['Bert_0']

test['Bert_1'] = Bert_test['Bert_1']
test['Bert_0'] = Bert_test['Bert_0']


# Adding Robert Probas to x2, x3, test

x2['Roberta_1'] = Roberta2['Roberta_1']
x2['Roberta_0'] = Roberta2['Roberta_0']

x3['Roberta_1'] = Roberta3['Roberta_1']
x3['Roberta_0'] = Roberta3['Roberta_0']

test['Roberta_1'] = Roberta_test['Roberta_1']
test['Roberta_0'] = Roberta_test['Roberta_0']

In [6]:
x2.shape

(1903, 16)

In [7]:
x2[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.3631,0.6369,0.418081,0.581919,0.578602,0.421398,0.536737,0.463263,0.54,0.46,0.456238,0.543762,0.388212,0.611788,0.188238,0.811762


In [8]:
x3[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.174052,0.825948,0.907352,0.092648,0.089012,0.910988,0.227634,0.772366,0.06,0.94,0.456238,0.543762,0.057976,0.942024,0.073738,0.926262


In [9]:
test[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.276864,0.723136,0.430683,0.569317,0.649411,0.350589,0.191533,0.808467,0.4,0.6,0.522109,0.477891,0.954075,0.045925,0.986153,0.013847


In [10]:
y2 = set2.target
y3 = set3.target

In [11]:
y3.value_counts()

0    1073
1     831
Name: target, dtype: int64

In [12]:
x2['Bert_1'][:5].values.tolist()

[0.3882122039794922,
 0.9852584600448608,
 0.4531148076057434,
 0.9874793887138368,
 0.1129818707704544]

In [13]:
def deci(r):
  return '%.4f'%r

In [14]:
x2.columns, x3.columns, test.columns

(Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'),
 Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'),
 Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'))

In [15]:
cols = x2.columns.tolist()

In [16]:
for i in cols:
  x2[i] = x2[i].apply(deci)
  x2[i] = x2[i].astype(float)

  x3[i] = x3[i].apply(deci)
  x3[i] = x3[i].astype(float)

  test[i] = test[i].apply(deci)
  test[i] = test[i].astype(float)

In [17]:
test[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.2769,0.7231,0.4307,0.5693,0.6494,0.3506,0.1915,0.8085,0.4,0.6,0.5221,0.4779,0.9541,0.0459,0.9862,0.0138


In [18]:
#!pip install -q category_encoders
#import category_encoders as ce

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear',penalty='l1', random_state = 1)
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.05,subsample = 0.01,min_samples_split = 5, min_samples_leaf = 5,max_depth= 1, random_state=1)
from sklearn import svm
svc = svm.SVC(kernel = 'linear',probability=True, random_state=1)
from sklearn.ensemble import ExtraTreesClassifier
xtra_tree = ExtraTreesClassifier(random_state=1)  
from sklearn.naive_bayes import BernoulliNB
bayes = BernoulliNB(binarize=0.4)

In [21]:
#model = Pipeline([('enc', ce.LeaveOneOutEncoder()),
#                 ('clf', lr ) ])

model = lr
model.fit(x2,y2)

print('LOGISTIC REGRESSION')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

LOGISTIC REGRESSION
*******************************************************
Rounded F1 SCORE on set-2 : 0.81
Exact F1 SCORE on set-2 =  0.8095238095238096
*******************************************************
Rounded F1 SCORE on set-3 : 0.82
Exact F1 SCORE on set-3 =  0.8185185185185184
*******************************************************

TP : 663 	FN : 168 
TN : 947 	FP : 126 


*******************************************************
              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1073
           1       0.84      0.80      0.82       831

    accuracy                           0.85      1904
   macro avg       0.84      0.84      0.84      1904
weighted avg       0.85      0.85      0.85      1904

*******************************************************


In [23]:
model = svc
model.fit(x2,y2)

print('SUPPORT VECTOR MACHINE')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

SUPPORT VECTOR MACHINE
*******************************************************
Rounded F1 SCORE on set-2 : 0.80
Exact F1 SCORE on set-2 =  0.7984084880636604
*******************************************************
Rounded F1 SCORE on set-3 : 0.81
Exact F1 SCORE on set-3 =  0.8147208121827413
*******************************************************

TP : 642 	FN : 189 
TN : 970 	FP : 103 


*******************************************************
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1073
           1       0.86      0.77      0.81       831

    accuracy                           0.85      1904
   macro avg       0.85      0.84      0.84      1904
weighted avg       0.85      0.85      0.85      1904

*******************************************************


In [24]:
model = xtra_tree 
model.fit(x2,y2)

print('EXTRA TREES CLASSIFIER')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

EXTRA TREES CLASSIFIER
*******************************************************
Rounded F1 SCORE on set-2 : 1.00
Exact F1 SCORE on set-2 =  0.9993876301285977
*******************************************************
Rounded F1 SCORE on set-3 : 0.80
Exact F1 SCORE on set-3 =  0.8034825870646767
*******************************************************

TP : 646 	FN : 185 
TN : 942 	FP : 131 


*******************************************************
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1073
           1       0.83      0.78      0.80       831

    accuracy                           0.83      1904
   macro avg       0.83      0.83      0.83      1904
weighted avg       0.83      0.83      0.83      1904

*******************************************************


In [25]:
model =  bayes 
model.fit(x2,y2)

print('BERNOULLI BAYES')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

BERNOULLI BAYES
*******************************************************
Rounded F1 SCORE on set-2 : 0.79
Exact F1 SCORE on set-2 =  0.7854050711193569
*******************************************************
Rounded F1 SCORE on set-3 : 0.79
Exact F1 SCORE on set-3 =  0.788242351529694
*******************************************************

TP : 657 	FN : 174 
TN : 894 	FP : 179 


*******************************************************
              precision    recall  f1-score   support

           0       0.84      0.83      0.84      1073
           1       0.79      0.79      0.79       831

    accuracy                           0.81      1904
   macro avg       0.81      0.81      0.81      1904
weighted avg       0.81      0.81      0.81      1904

*******************************************************


In [26]:
!pip install -q catboost

     |████████████████████████████████| 64.8MB 95kB/s 


In [27]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(iterations=3,
                           learning_rate= 0.05,
                           depth=1)

In [28]:
model =  catboost 
model.fit(x2,y2)

print('CATBOOST')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

0:	learn: 0.6718964	total: 48.8ms	remaining: 97.7ms
1:	learn: 0.6533466	total: 49.7ms	remaining: 24.9ms
2:	learn: 0.6344125	total: 50.4ms	remaining: 0us
CATBOOST
*******************************************************
Rounded F1 SCORE on set-2 : 0.80
Exact F1 SCORE on set-2 =  0.7956551255940258
*******************************************************
Rounded F1 SCORE on set-3 : 0.81
Exact F1 SCORE on set-3 =  0.8135152696556205
*******************************************************

TP : 626 	FN : 205 
TN : 991 	FP : 82 


*******************************************************
              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1073
           1       0.88      0.75      0.81       831

    accuracy                           0.85      1904
   macro avg       0.86      0.84      0.84      1904
weighted avg       0.85      0.85      0.85      1904

*******************************************************


In [38]:
model = gb
model.fit(x2,y2)

print('GRADIENT BOOSTING CLASSIFIER')
print('*'*55)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

GRADIENT BOOSTING CLASSIFIER
*******************************************************
Rounded F1 SCORE on set-2 : 0.82
Exact F1 SCORE on set-2 =  0.8168659534298301
*******************************************************
Rounded F1 SCORE on set-3 : 0.82
Exact F1 SCORE on set-3 =  0.8188405797101449
*******************************************************

TP : 678 	FN : 153 
TN : 926 	FP : 147 


*******************************************************
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1073
           1       0.82      0.82      0.82       831

    accuracy                           0.84      1904
   macro avg       0.84      0.84      0.84      1904
weighted avg       0.84      0.84      0.84      1904

*******************************************************


In [40]:
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))

Exact F1 SCORE on set-3 =  0.8188405797101449


In [ ]:
y3[10:20].tolist()

In [ ]:
pred[10:20] #0.3

In [ ]:
pred_proba[10:20] # 0,1 # 1st col: non-disaster proba; 2nd col: Disaster proba

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(model.predict_proba(test)[:,0])

In [ ]:
plt.hist(model.predict_proba(test)[:,1])

In [ ]:
pred


# ADJUSTING THE THRESHOLD

In [41]:
from sklearn.preprocessing import binarize

In [42]:

def all_metrics(n):
  pred = binarize(pred_proba,n)
  predictions = pred[:,1]
  predictions = predictions.astype(int)
  print('F1 SCORE rounded : %.2f'%f1_score(y3, predictions, average='binary'))
  print('\n\nF1 SCORE = ' ,f1_score(y3, predictions, average='binary'))
  tn, fp, fn, tp = confusion_matrix(y3, predictions).ravel()
  print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
  print('*'*55)

  print(classification_report(y3, predictions))
  print('*'*55)

# DEFAULT THRESHOLD : 0.5

In [43]:
all_metrics(0.5) 

F1 SCORE rounded : 0.82


F1 SCORE =  0.8188405797101449

TP : 678 	FN : 153 
TN : 926 	FP : 147 


*******************************************************
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1073
           1       0.82      0.82      0.82       831

    accuracy                           0.84      1904
   macro avg       0.84      0.84      0.84      1904
weighted avg       0.84      0.84      0.84      1904

*******************************************************


# FINDING THE THRESHOLD WITH THE BEST F1 SCORE


In [44]:
def threshold_selection(n):
  pred = binarize(pred_proba,n)
  predictions = pred[:,1]
  predictions = predictions.astype(int)
  
  return f1_score(y3, predictions, average='binary')
  


In [45]:
q = threshold_selection(0.5)
q

0.8188405797101449

In [46]:
import numpy as np

# Loop through 100 numbers from 0.1 to 1.0 to find the best threshold

In [47]:
f1 = 0.8185185185185184
t = 0.5
# add for loop 0.1-0.99 ; find the value of n for high overall f1 score
for n in np.arange(0.1,1.0,0.01):
  if threshold_selection(n) > f1:
    f1 = threshold_selection(n)
    t = n

print(f1)
print(t)

0.8214285714285714
0.6699999999999997


In [48]:
t1 = threshold_selection(0.5)
t2 = threshold_selection(0.67)

print(t1)
print(t2)


0.8188405797101449
0.8214285714285714


# NEW THRESHOLD


In [49]:
all_metrics(0.67)

F1 SCORE rounded : 0.82


F1 SCORE =  0.8214285714285714

TP : 644 	FN : 187 
TN : 980 	FP : 93 


*******************************************************
              precision    recall  f1-score   support

           0       0.84      0.91      0.88      1073
           1       0.87      0.77      0.82       831

    accuracy                           0.85      1904
   macro avg       0.86      0.84      0.85      1904
weighted avg       0.85      0.85      0.85      1904

*******************************************************


In [55]:
y3[:5]

0    0
1    1
2    1
3    0
4    1
Name: target, dtype: int64

In [56]:
pred_proba[:5]

array([[0.93224736, 0.06775264],
       [0.77537829, 0.22462171],
       [0.08555151, 0.91444849],
       [0.88049244, 0.11950756],
       [0.07405367, 0.92594633]])

In [57]:
pred[:5]

array([0, 0, 1, 0, 1])

# CHANGING THE THRESHOLD TO 0.67

In [58]:
test_proba

array([[0.05935987, 0.94064013],
       [0.06475144, 0.93524856],
       [0.08691015, 0.91308985],
       ...,
       [0.06275819, 0.93724181],
       [0.0512621 , 0.9487379 ],
       [0.13439484, 0.86560516]])

In [59]:
test_proba_new = binarize(test_proba,0.67)

In [60]:
test_proba_new[:2]

array([[0., 1.],
       [0., 1.]])

In [61]:

predictions = test_proba_new[:,1]
predictions[:10]

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [62]:
predictions = predictions.astype(int)
predictions[:10]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [63]:
predictions = predictions.tolist()
len(predictions)

3263

# Load Test Data

In [64]:
submission = pd.read_csv('/content/drive/My Drive/Tweets/2. Input/tweets_sample_submission.csv')

In [65]:
output = pd.DataFrame({'id': submission.id, 'target': predictions})

In [66]:
output[:5]

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [67]:
output.to_csv('submission2.csv', index=False) 
!cp submission2.csv "/content/drive/My Drive/Tweets/7. Output" 